In [ ]:
import pandas as pd
import os
import tensorflow as tf
import tensorflow_hub as hub
!pip install tensorflow_text
import tensorflow_text as text
!pip install tensorflow_addons
import tensorflow_addons as tfa
!pip install -q tf-models-official
from official.nlp import optimization  # to create AdamW optmizer
from keras.callbacks import ModelCheckpoint
import numpy as np

     |████████████████████████████████| 3.4MB 7.8MB/s 
     |████████████████████████████████| 706kB 8.4MB/s 
     |████████████████████████████████| 1.1MB 7.7MB/s 
     |████████████████████████████████| 358kB 22.6MB/s 
     |████████████████████████████████| 102kB 10.0MB/s 
     |████████████████████████████████| 645kB 29.7MB/s 
     |████████████████████████████████| 174kB 35.3MB/s 
     |████████████████████████████████| 1.2MB 32.7MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 37.6MB 90kB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) 

Mounted at /content/drive


In [ ]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'
bert_model_name = 'bert_multi_cased_L-12_H-768_A-12'
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12'

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12': 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12': 'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12': 'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base': 'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small': 'https://tfhub.dev/google/electra_small/2',
    'electra_base': 'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed': 'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books': 'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base': 'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12': 'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12': 'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base': 'https://tfhub.dev/tensorflow/albert_en_preprocess/2',
    'electra_small': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [ ]:
os.chdir('/content/drive/MyDrive/Capgemini/Hackathon/notebooks')
cleaned_df = pd.read_csv('../data/all/400k.csv', sep='#', index_col=[0])
X_test = pd.read_csv('../data/original/test.csv', index_col=['id'])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
cleaned_df.dropna(subset=['description'], inplace=True)
two_or_more = cleaned_df[cleaned_df['len_title'] >= 2]
two_or_more = two_or_more[two_or_more['len_description'] >= 2]
two_or_more.drop_duplicates(subset=['title'], inplace=True)
two_or_more.drop_duplicates(subset=['description'], inplace=True)
subset = two_or_more.groupby(by=['class']).sample(n=10000)
print(subset.shape)
subset_random = subset.sample(frac=1, random_state=42)
print(subset_random.head())

(40000, 5)
        class  ... len_title
99199       3  ...      10.0
289944      1  ...      12.0
369938      2  ...      12.0
43131       0  ...       8.0
366509      2  ...       8.0

[5 rows x 5 columns]


In [ ]:
subset_random['concatenation'] = subset_random['title'] + subset_random['description']
X_test['description'].fillna('', inplace=True)
X_test['concatenation'] = X_test['title'] + X_test['description']

In [ ]:
X_test.head()

,title,description,concatenation
id,,,
0,PDP 048-121-NA Afterglow Wired Controller for ...,Enhance your gaming experience with the Afterg...,PDP 048-121-NA Afterglow Wired Controller for ...
1,Gold Medal Groomers Ear Powder (30 Grams),"Great for relieving irritated itchy ears, Groo...",Gold Medal Groomers Ear Powder (30 Grams)Great...
2,Sports Research Sweet Sweat Premium Waist Trim...,,Sports Research Sweet Sweat Premium Waist Trim...
3,Gildan Men's Fleece Open Bottom Pocketed Pant,Gildan is one of the world's largest verticall...,Gildan Men's Fleece Open Bottom Pocketed PantG...
4,Sennheiser GAME ZERO Gaming Headset- Black,The new closed back G4ME ZERO headset is ideal...,Sennheiser GAME ZERO Gaming Headset- BlackThe ...


In [ ]:
def load_data(Xy_train, X_test, column):

    Xy_train_column = Xy_train.dropna(subset=[column], inplace=False)
    X_test_column = X_test.dropna(subset=[column], inplace=False)
    X_test_column = X_test_column[column]
    X_train_column = Xy_train_column[column]
    y_train_column = Xy_train_column['class']
    y_train_column_cat = tf.keras.utils.to_categorical(y_train_column)

    return X_train_column, y_train_column_cat, X_test_column

In [ ]:
def train_model(X_train, y_train_cat, tfhub_handle_encoder, tfhub_handle_preprocess, model_name, seq_length=172, epochs=15):

    def build_classifier_model(seq_length):
        text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
        preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
        encoder_inputs = preprocessing_layer(text_input)
        encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
        outputs = encoder(encoder_inputs)
        net = outputs['pooled_output']
        net = tf.keras.layers.Dropout(0.1)(net)
        net = tf.keras.layers.Dense(4, activation=tf.keras.activations.softmax, name='classifier')(net)
        return tf.keras.Model(text_input, net)

    bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
    bert_model = hub.KerasLayer(tfhub_handle_encoder)

    classifier_model = build_classifier_model(seq_length)
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    metrics = tfa.metrics.F1Score(num_classes=4)

    steps_per_epoch = len(X_train)
    num_train_steps = steps_per_epoch * epochs
    num_warmup_steps = int(0.1 * num_train_steps)

    init_lr = 3e-5
    optimizer = optimization.create_optimizer(init_lr=init_lr,
                                            num_train_steps=num_train_steps,
                                            num_warmup_steps=num_warmup_steps,
                                            optimizer_type='adamw')
    
    classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
    
    checkpoint = ModelCheckpoint(model_name + ".hdf5", monitor='val_f1_score', verbose=1,
                                 save_best_only=True, mode='auto', period=1)

    print(f'Training model with {tfhub_handle_encoder}')
    history = classifier_model.fit(X_train, y_train_cat, 32, epochs=epochs, validation_split=0.20)

    return classifier_model, optimizer

In [ ]:
def predict_and_save(classifier_model, X_test, filename_submission, filename_unpack, filename_model, labels):

    y_pred_cat = classifier_model.predict(
        X_test, batch_size=None, verbose=1, steps=None, callbacks=None, max_queue_size=10,
        workers=1, use_multiprocessing=False)
    
    y_pred = np.argmax(y_pred_cat, axis=1)

    y_pred_submission = pd.DataFrame(y_pred, index=X_test.index.values, columns=['class'])
    y_pred_submission.index.rename('id', inplace=True)
    y_pred_submission.to_csv(os.path.join('..', 'data', filename_submission))

    y_pred_cat_df = pd.DataFrame(y_pred_cat, index=X_test.index.values, columns=labels)
    y_pred_cat_df.index.rename('id', inplace=True)
    y_pred_cat_df.to_csv(os.path.join('..', 'data', filename_unpack))

    classifier_model.save(os.path.join('..', 'saved_models', 'voting_classifier', filename_model))

In [ ]:
X_train_concat, y_train_concat_cat, X_test_concat = load_data(subset_random, X_test, 'concatenation')
classifier_model_concat, optimizer_concat = train_model(X_train_concat, y_train_concat_cat, tfhub_handle_encoder, tfhub_handle_preprocess, epochs=6, model_name='checkpoint_big_data_concat')
predict_and_save(classifier_model_concat, X_test_concat, 'concat_big_data.csv', 'concat_softmax_big_data.csv', 'concat_big_data', labels=['concat_0', 'concat_1', 'concat_2', 'concat_3'])

Training model with https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Epoch 1/6
1000/1000 [==============================] - 643s 621ms/step - loss: 1.2575 - f1_score: 0.4071 - val_loss: 0.2605 - val_f1_score: 0.9219
Epoch 2/6
1000/1000 [==============================] - 622s 622ms/step - loss: 0.2404 - f1_score: 0.9270 - val_loss: 0.1894 - val_f1_score: 0.9418
Epoch 3/6
1000/1000 [==============================] - 623s 623ms/step - loss: 0.1644 - f1_score: 0.9498 - val_loss: 0.1665 - val_f1_score: 0.9518
Epoch 4/6
1000/1000 [==============================] - 623s 623ms/step - loss: 0.1322 - f1_score: 0.9620 - val_loss: 0.1729 - val_f1_score: 0.9518
Epoch 5/6
1000/1000 [==============================] - 623s 623ms/step - loss: 0.1021 - f1_score: 0.9723 - val_loss: 0.1737 - val_f1_score: 0.9577
Epoch 6/6
53/53 [==============================] - 13s 233ms/step


NameError: ignored